In [23]:
!pip install -U "datasets==3.2.0"
!pip install -U "evaluate" "transformers" "sentencepiece"
!pip install bert-score sacrebleu rouge-score sumy indic-nlp-library


# Import Libraries

In [24]:
import re
import torch
import nltk
from datasets import load_dataset
from indicnlp.tokenize import indic_tokenize
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from rouge_score import rouge_scorer
from bert_score import score as bert_scorer
from sacrebleu.metrics import CHRF


In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Preprocessing

In [26]:
TELUGU_STOPWORDS = {'మరుగు','లో','కు','ఈ','అని','ది','తో','కోసం','ఇది','వద్ద','పై','అది'}
ENGLISH_STOPWORDS = set(stopwords.words('english'))

def preprocess_bilingual_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[“”‘’"\'’]', '', text)
    text = re.sub(r'([?.!,])', r' \1 ', text)
    tokens = indic_tokenize.trivial_tokenize(text)

    telugu_tokens, english_tokens = [], []

    for token in tokens:
        if re.search(r'[\u0C00-\u0C7F]', token):
            if token not in TELUGU_STOPWORDS:
                telugu_tokens.append(token)
        else:
            token = token.lower()
            if token not in ENGLISH_STOPWORDS:
                english_tokens.append(token)

    return ' '.join(telugu_tokens + english_tokens)


# Load Dataset

In [27]:
dataset = load_dataset("csebuetnlp/xlsum", "telugu")

sample = dataset['train'][0]
original_text = sample['text']
reference_summary_te = sample['summary']

print("Original (Telugu):", original_text[:300])
cleaned_text = preprocess_bilingual_text(original_text)
print("\nCleaned bilingual (extractive use only):", cleaned_text[:300])


Original (Telugu): అయితే, ఆ జాబితా తప్పులతడకని పైలట్లు అంటున్నారు. విమర్శలు రావడంతో లైసెన్సులు నకిలీవి కాదని, పరీక్ష ప్రక్రియలో లోపోలున్నాయని ప్రభుత్వం పేర్కొంది. ఇమ్రాన్‌ఖాన్ ప్రభుత్వం దిద్దుబాటు చర్యలు తీసుకుంటుండగా.. లైసెన్స్ స్కామ్‌తో సంబంధం ఉన్న వారిపై చర్యలు తీసుకోవాలని సుప్రీంకోర్టు ఆదేశించింది. బీబీసీ ప్రతినిధ

Cleaned bilingual (extractive use only): అయితే ఆ జాబితా తప్పులతడకని పైలట్లు అంటున్నారు విమర్శలు రావడంతో లైసెన్సులు నకిలీవి కాదని పరీక్ష ప్రక్రియలో లోపోలున్నాయని ప్రభుత్వం పేర్కొంది ఇమ్రాన్‌ఖాన్ ప్రభుత్వం దిద్దుబాటు చర్యలు తీసుకుంటుండగా లైసెన్స్ స్కామ్‌తో సంబంధం ఉన్న వారిపై చర్యలు తీసుకోవాలని సుప్రీంకోర్టు ఆదేశించింది బీబీసీ ప్రతినిధి షుమాయ


# Extractive Summaries

# TF-based extractive

In [28]:
def tf_extractive_summary(text, top_k=3):
    sentences = re.split(r'(?<=[\\.।])\s+', text)
    if len(sentences) == 1:
        sentences = text.split('. ')
    tokens = [w for s in sentences for w in s.split()]
    freq = {}
    for t in tokens:
        freq[t] = freq.get(t, 0) + 1

    maxf = max(freq.values()) if freq else 1
    for k in freq:
        freq[k] /= maxf

    sent_scores = [(s, sum(freq.get(w,0) for w in s.split())) for s in sentences]
    top = sorted(sent_scores, key=lambda x:x[1], reverse=True)[:top_k]
    summary = ' '.join(s[0].strip() for s in sorted(top, key=lambda x:sentences.index(x[0])))
    return summary


# TextRank

In [29]:
def textrank_summary(text, sentence_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer('english'))
    summarizer = TextRankSummarizer()
    summary_sentences = summarizer(parser.document, sentence_count)
    return ' '.join(str(s) for s in summary_sentences)


# Load multilingual mT5 model , Translation Model
# Creating Translation Pipeline

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device =", device)

# Telugu Summarization model
summ_model_name = "csebuetnlp/mT5_multilingual_xlsum"
summ_tokenizer = AutoTokenizer.from_pretrained(summ_model_name)
summ_model = AutoModelForSeq2SeqLM.from_pretrained(summ_model_name).to(device)

# Translation model NLLB
tran_model_name = "facebook/nllb-200-distilled-600M"
tran_tokenizer = AutoTokenizer.from_pretrained(tran_model_name)
tran_model = AutoModelForSeq2SeqLM.from_pretrained(tran_model_name).to(device)

translator = pipeline(
    "translation",
    model=tran_model,
    tokenizer=tran_tokenizer,
    src_lang="tel_Telu",
    tgt_lang="eng_Latn",
    device=0 if device == "cuda" else -1
)


Device = cuda


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


# Generate Abstractive Summary

In [31]:
def generate_telugu_summary(text):
    inputs = summ_tokenizer(text, return_tensors="pt", truncation=True).to(device)
    summary_ids = summ_model.generate(inputs["input_ids"], max_length=60, min_length=10)
    return summ_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

telugu_summary = generate_telugu_summary(original_text)
print("\n Telugu Summary:", telugu_summary)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



 Telugu Summary: పాకిస్తాన్ ప్రధాని ఇమ్రాన్ ఖాన్ ప్రభుత్వం ఇటీవల జారీ చేసిన లైసెన్స్ పరీక్షల జాబితాపై తీవ్ర విమర్శలు వెల్లువెత్తాయి.


# Translate Summary to English and Reference Summary

In [32]:

english_summary = translator(telugu_summary)[0]['translation_text']
print("\n English Summary:", english_summary)

reference_summary_en = translator(reference_summary_te)[0]["translation_text"]
print("\n Reference English Summary:", reference_summary_en)





 English Summary: The recent list of licensure examinations issued by the government of Prime Minister Imran Khan of Pakistan has been heavily criticized.

 Reference English Summary: A new crisis has begun in Pakistan's aviation industry, with the airline's own aviation minister releasing a list of 262 pilots with fake licenses.



# BERTScore and ChrF Score

In [33]:

P, R, F1 = bert_scorer([english_summary], [reference_summary_en], lang="en")
print("\n BERTScore F1:", float(F1[0]))

chrf = CHRF()
chrf_score = chrf.sentence_score(english_summary, [reference_summary_en])
print("\n ChrF Score:", chrf_score.score)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 BERTScore F1: 0.8884020447731018

 ChrF Score: 28.88525305218867


# Example

In [35]:
# ✅ Telugu long text
telugu_text = """
పాకిస్తాన్ విమానాయాన రంగంలో భారీ వివాదం చెలరేగింది.పాకిస్తాన్ ప్రభుత్వ ప్రకటన ప్రకారం దేశంలోని 860 మంది పైలట్లలో 262 మంది పైలట్లు నకిలీ లైసెన్సులు కలిగి ఉన్నట్లు ఆరోపణలు వచ్చాయి.
ఈ విషయంపై పార్లమెంట్‌లో షాక్‌కు గురిచేసే వివరాలు వెల్లడికాగా ప్రముఖ ఎయిర్‌లైన్లలో పనిచేస్తున్న పలు పైలట్లు పరీక్షలు రాయకుండాలైసెన్సులు పొందినట్లు సమాచారం.ఈ ఘటన ప్రపంచవ్యాప్తంగా విమానయాన భద్రతపై తీవ్ర ఆందోళనను రేపుతోంది.
"""

# ✅ Telugu reference summary
ref_telugu_summary = """
పాకిస్తాన్‌లో 262 మంది పైలట్లు నకిలీ లైసెన్సులు కలిగి ఉన్నారని ఆరోపణలు రావడంతో విమానయాన రంగంలో భారీ వివాదం చెలరేగింది.
"""

# ✅ English reference summary
ref_english_summary = """
A major controversy erupted in Pakistan after allegations surfaced that 262 pilots
held fake licenses, causing global concern over aviation safety.
"""

generated_telugu_summary = generate_telugu_summary(telugu_text)
generated_english_summary = translator(generated_telugu_summary)[0]['translation_text']

print("Generated Telugu Summary:", generated_telugu_summary)
print("Generated English Summary:", generated_english_summary)


Generated Telugu Summary: పాకిస్తాన్ విమానయాన రంగంలో నకిలీ లైసెన్సులు కలిగి ఉన్నట్లు ఆరోపణలు ఎదుర్కొంటున్న పైలట్ల వివరాలను పార్లమెంట్ లో వెల్లడించింది.
Generated English Summary: The Parliament has unveiled details of pilots facing allegations of having fake licenses in Pakistan's aviation sector.
